## MIDBench API Demo Code for Windows Users

In [1]:
# Check Ubuntu Release
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.5 LTS
Release:	18.04
Codename:	bionic


In [ ]:
# Install System Dependencies
!sudo apt-get update && sudo apt-get install -y \
    build-essential \
    libssl-dev \
    uuid-dev \
    libgpgme11-dev \
    squashfs-tools \
    libseccomp-dev \
    wget \
    pkg-config \
    git \
    cryptsetup

In [ ]:
# Install Go
%%bash
wget https://go.dev/dl/go1.18.4.linux-amd64.tar.gz && \
sudo rm -rf /usr/local/go && tar -C /usr/local -xzf go1.18.4.linux-amd64.tar.gz

In [ ]:
# Download Singularity
%%bash
export VERSION=3.5.3 && # adjust this as necessary \
  wget https://github.com/singularityware/singularity/releases/download/v${VERSION}/singularity-${VERSION}.tar.gz && \
  tar -xzf singularity-${VERSION}.tar.gz && \
  cd singularity

In [ ]:
# Compile Singularity Source Code
%%shell
export PATH=$PATH:/usr/local/go/bin && \
source /etc/profile && \
go version && \
cd singularity && \
./mconfig && \
    make -C builddir && \
    sudo make -C builddir install

In [ ]:
# Verify Singularity Installation
!singularity version

In [ ]:
# Pull SU2 Singularity Container
!singularity pull --arch amd64 library://junideallab/midbench/su2v7.3.1_conda3.9.12_gmsh:latest

In [ ]:
# Download and Unzip MIDBenchmarkSuite code
!unzip MIDBenchmarkSuite.zip

In [ ]:
# Create Python Script for 2D Aifoil Simulation Demo Implementation
%%writefile MIDBenchmarkSuite/airfoil2d_simu.py
from midbench.envs import make 
Env, Design, Condition = make("Airfoil2d-v0")

designs = Design('./midbench/envs/airfoil/airfoils_pred_cbegan_example.npy').meshgen() 
conditions = Condition(**{'mach':0.7,'reynolds':7000000,'lift':0.350}) 
performances = ['lift', 'drag'] 

lift, drag = Env.simulate(conditions, designs, performances, './midbench/envs/airfoil/results_simu') 
print(lift, drag)

In [ ]:
# Create Python Script for 2D Aifoil Optimization Demo Implementation
%%writefile MIDBenchmarkSuite/airfoil2d_opt.py
from midbench.envs import make 
Env, Design, Condition = make("Airfoil2d-v0")

designs = Design('./midbench/envs/airfoil/airfoils_pred_cbegan_example.npy')
conditions = Condition(**{'mach':0.6,'reynolds':8000000,'lift':0.320})
objectives = ['drag', 'ld_ratio', 'airfoil_opt']

drag, ld_ratio, airfoil_opt = Env.optimize(conditions, designs, objectives, './midbench/envs/airfoil/results_opt')
print(drag, ld_ratio, airfoil_opt)

In [ ]:
# Execute Simualtion Script using SU2 Singularity Container
%%bash
cd MIDBenchmarkSuite/
singularity exec --userns ../su2v7.3.1_conda3.9.12_gmsh_latest.sif bash -c "python airfoil2d_simu.py"

In [ ]:
# Execute Optimization Script using SU2 Singularity Container
%%bash
cd MIDBenchmarkSuite/
singularity exec --userns ../su2v7.3.1_conda3.9.12_gmsh_latest.sif bash -c "python airfoil2d_opt.py"